In [1]:
# Initialization vector (IV) is typically a public random string that is used as a third input—in addition to the key and plaintext—into the encryption algorithm
# The IV is supposed to be different each time you encrypt, preventing the same data from being encrypted to the same ciphertext
# PKCS7 padding appends n bytes, with each padding byte holding the value n: if 3 bytes of padding are needed, it appends \x03\x03\x03. Similarly, if 2 bytes of padding are needed, it appends \x02\x02.
# ANSI X.923 padding all appended bytes are 0, except for the last byte, which is the length of the total padding. In this example, 3 bytes of padding is \x00\x00\x03, and two bytes of padding is \x00\x02.
#
# If you reuse a key and IV pair, you will get predictable output for predictable headers. Parts of your messages that you might be inclined not to think about at all,
# because they are boilerplate or contain hidden structure, will become a liability; adversaries can use predictable ciphertext to learn about your keys.
#

In [2]:
# For example  cipher block chaining (CBC) mode:
# P'[n] = P[n]*C[n-1]
# where
# * := xor
# P[n] := plaintext
# C[n-1] := cyphertext from previous block
# P'[n] := plaintext combined with previous cypher text
#
# To get back the plaintext  do the following
# P'[n] = P[n]*C[n-1]
# C[n-1]*P[n] = P'[n]
# C[n-1]*C[n-1]*P[n] = C[n-1]*P'[n]
# P[n] = C[n-1]*P'[n]

In [ ]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
import os

# never reuse key and IV pairs.
# Always use a new key/IV pair when encrypting anything.
# For CBC, if you reuse a key and IV pair, you will get predictable output for predictable headers.
key = os.urandom(32)
# The IVs are always 128 bits long (16 bytes), even if the key size is larger (typically 196 or 256 bits)
iv = os.urandom(16)

aesCipher = Cipher(algorithms.AES(key),
                   modes.CBC(iv),
                   backend=default_backend())
aesEncryptor = aesCipher.encryptor()
aesDecryptor = aesCipher.decryptor()

In [ ]:
# AES (Advanced Encryption Standard) Counter Mode (CTR) is a method of encrypting data that turns a block cipher into a stream cipher.
# In CTR mode, instead of encrypting the actual data block by block, an incrementing counter value is encrypted and then XORed with the
# plaintext to produce the ciphertext. The counter is usually combined with an Initialization Vector (IV) to ensure uniqueness.

# Here's how it works:

# An IV is chosen. This can be any number, but it should be unique for each message encrypted with the same key.
# A counter is initialized. This is typically just a number that starts at zero and increments for each block of data.
# For each block of data, the block cipher encrypts the combination of the IV and the current counter value.
# The encrypted IV/counter value is XORed with the plaintext block to produce the ciphertext block.
# The counter is incremented and the process is repeated for the next block of data.
# The reason why you should never reuse the same IV and key pair with AES CTR is that it can lead to serious security vulnerabilities. 
# If an attacker gets two ciphertexts that were encrypted with the same key and IV, they can XOR the two ciphertexts together. 
# Because the same keystream was used to encrypt both plaintexts, the XOR of the two ciphertexts is equal to the XOR of the two plaintexts. 
# This can give the attacker information about the relationship between the two plaintexts, and in some cases, might allow the attacker to recover the plaintexts entirely.

# This is not a problem unique to AES CTR; it's a general principle of all stream ciphers and modes of operation that turn block ciphers into stream ciphers.
# Always make sure to use a unique IV for each message encrypted under the same key. This can often be accomplished by including 
# a nonce (number used once) in the IV or using a counter as part of the IV.

In [3]:
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding

# Key and IV generation
key = os.urandom(32)
iv = os.urandom(16)

# Define fixed length header and variable length message
header = "This is a header of 32 bytes."
message = "Hello, this is a secret message."

# Concatenate header and message
plaintext = header + message

# Create cipher object
aesCipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())

# Pad the plaintext message to ensure it's a multiple of the block size for CBC
padder = padding.PKCS7(128).padder()
padded_plaintext = padder.update(plaintext.encode()) + padder.finalize()

# Encryption
aesEncryptor = aesCipher.encryptor()
ciphertext = aesEncryptor.update(padded_plaintext) + aesEncryptor.finalize()

# Decryption
aesDecryptor = aesCipher.decryptor()
decryptedtext = aesDecryptor.update(ciphertext) + aesDecryptor.finalize()

# Unpadding the plaintext message
unpadder = padding.PKCS7(128).unpadder()
decrypted_plaintext = unpadder.update(decryptedtext) + unpadder.finalize()

print("Original message: ", plaintext)
print("Decrypted message: ", decrypted_plaintext.decode())


Original message:  This is a header of 32 bytes.Hello, this is a secret message.
Decrypted message:  This is a header of 32 bytes.Hello, this is a secret message.


In [5]:
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding

# Key and IV generation
key = os.urandom(32)
iv = os.urandom(16)

# Define fixed length header and variable length message
header = "This is a header of 32 bytes."
message1 = "Hello, this is a secret message."
message2 = "This is another secret message."

# Concatenate header and messages
plaintext1 = header + message1
plaintext2 = header + message2

# Create cipher objects
aesCipher1 = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
aesCipher2 = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())

# Pad the plaintext messages to ensure they're multiples of the block size for CBC
padder1 = padding.PKCS7(128).padder()
padded_plaintext1 = padder1.update(plaintext1.encode()) + padder1.finalize()

padder2 = padding.PKCS7(128).padder()
padded_plaintext2 = padder2.update(plaintext2.encode()) + padder2.finalize()

# Encryption
aesEncryptor1 = aesCipher1.encryptor()
aesEncryptor2 = aesCipher2.encryptor()
ciphertext1 = aesEncryptor1.update(padded_plaintext1) + aesEncryptor1.finalize()
ciphertext2 = aesEncryptor2.update(padded_plaintext2) + aesEncryptor2.finalize()

# Print both ciphertexts
print("Ciphertext 1: ", ciphertext1)
print("Ciphertext 2: ", ciphertext2)

# If the first block of ciphertext1 and ciphertext2 are the same, it indicates that 
# the same plaintext header was encrypted with the same key and IV, which is not secure.

# Decryption
aesDecryptor1 = aesCipher1.decryptor()
aesDecryptor2 = aesCipher2.decryptor()
decryptedtext1 = aesDecryptor1.update(ciphertext1) + aesDecryptor1.finalize()
decryptedtext2 = aesDecryptor2.update(ciphertext2) + aesDecryptor2.finalize()

# Unpadding the plaintext messages
unpadder1 = padding.PKCS7(128).unpadder()
decrypted_plaintext1 = unpadder1.update(decryptedtext1) + unpadder1.finalize()

unpadder2 = padding.PKCS7(128).unpadder()
decrypted_plaintext2 = unpadder2.update(decryptedtext2) + unpadder2.finalize()

print("Original message 1: ", plaintext1)
print("Decrypted message 1: ", decrypted_plaintext1.decode())
print("Original message 2: ", plaintext2)
print("Decrypted message 2: ", decrypted_plaintext2.decode())


Ciphertext 1:  b'&\xdc\rpo\xc5pGn\xa8t;c\x93\xb3\x8d\x03n\xa7\x18k\x9b\xac\xef\x14\xf1i\x87M\x81p\x13\xa4\x16\xf7\xaca \xa3Q\xbc\x91`\x94A\xe7\n\x92\x01L\xf2\xfd\x06\xe0S|\xa1\xd1. jt\x852'
Ciphertext 2:  b'&\xdc\rpo\xc5pGn\xa8t;c\x93\xb3\x8dm\x06n;r]O\x05\x89\xd8s\xd3\xe6\xd8\x8cY\xc7\x96CuO\xa7R\xebS}Wa^\x01oO64\xf5\xcd\n\xa3\x81\xefM\xe7dCk~7\x98'
Original message 1:  This is a header of 32 bytes.Hello, this is a secret message.
Decrypted message 1:  This is a header of 32 bytes.Hello, this is a secret message.
Original message 2:  This is a header of 32 bytes.This is another secret message.
Decrypted message 2:  This is a header of 32 bytes.This is another secret message.


In [2]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding
import os

# The secret key
key = os.urandom(32)

# Create a plaintext message
message = "This is a secret message."

# Function to encrypt a message
def encrypt_message(message, key):
    # Generate a random IV
    iv = os.urandom(16)

    # Create a cipher object
    aesCipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())

    # Padding
    padder = padding.PKCS7(128).padder()
    padded_message = padder.update(message.encode()) + padder.finalize()

    # Encryption
    aesEncryptor = aesCipher.encryptor()
    ciphertext = aesEncryptor.update(padded_message) + aesEncryptor.finalize()

    return iv + ciphertext

# Function to decrypt a message
def decrypt_message(ciphertext, key):
    # Separate the IV from the ciphertext
    iv = ciphertext[:16]
    ciphertext = ciphertext[16:]

    # Create a cipher object
    aesCipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())

    # Decryption
    aesDecryptor = aesCipher.decryptor()
    decryptedtext = aesDecryptor.update(ciphertext) + aesDecryptor.finalize()

    # Unpadding
    try:
        unpadder = padding.PKCS7(128).unpadder()
        plaintext = unpadder.update(decryptedtext) + unpadder.finalize()
        return plaintext
    except:
        return "Padding Error"

# Encrypt the message
ciphertext = encrypt_message(message, key)

# Decrypt the message
print(decrypt_message(ciphertext, key))

# The Padding Oracle Attack

# Change one byte in the ciphertext
ciphertext = bytearray(ciphertext)
ciphertext[16] ^= 0x01  # flip a bit in the first block

# Decrypt the modified ciphertext
print(decrypt_message(ciphertext, key))  # Padding Error

# TODO! Finish doc Exploiting Malleability

b'This is a secret message.'
b'\xad\xbb\xe2\x9cJ\xab,\xbc\xcb\xc2\x17\x1a\x1a\xf2\xbe\xda!message.'


In [2]:
# Ref: https://research.nccgroup.com/2021/02/17/cryptopals-exploiting-cbc-padding-oracles/
# Ref: https://medium.com/@c0D3M/poodle-attack-explained-ed6a1cd0667d